In [1]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [50]:
def professor_link_info(url_class):
    query_part = '?page='
    pages = 58

    all_docs = []

    for i in range(0, pages):
        # Load, chunk and index the contents of the blog.
        link = "https://wit.edu/faculty-staff-directory" + query_part + str(i + 1)
        loader = WebBaseLoader(
            web_paths=(link,),
            bs_kwargs=dict(
                parse_only=bs4.SoupStrainer(
                    class_=(url_class)
                )
            ),
        )
        loaded_doc = loader.load()
        all_docs.extend(loaded_doc)
        
    return all_docs

In [64]:
professor_titles = professor_link_info("profile-teaser__info__title")
professor_digital_contact = professor_link_info("profile-teaser__info__contact-digital")
professor_physical_contact = professor_link_info("profile-teaser__info__contact-physical")

In [65]:
print(len(professor_titles))
print(len(professor_digital_contact))
print(len(professor_physical_contact))

58
58
58


In [63]:
professor_titles

[Document(page_content='', metadata=''),
 Document(page_content="\n\nDanny Avelar\n\n\n\nMaintenance A Worker, Physical Plant\n\n\n\n\nAnthony Averbeck\n\n\n\nAdjunct Instructor, School of Architecture and Design\n\n\n\n\nFederica Aveta\n\n\n\nRegular Track, Electrical and Computer Engineering, School of Engineering\n\n\n\n\nAyla Axeloons\n\n\n\nCoach, Athletics\n\n\n\n\nJessy Ayala\n\n\n\nAdjunct Instructor, School of Computing and Data Science\n\n\n\n\nRolf Backmann\n\n\n\nFaculty - Adjunct, Architecture, School of Architecture and Design\n\n\n\n\nWilliam Bagnall\n\n\n\nAdmissions Counselor, Admissions\n\n\n\n\nJamillah Bailey\n\n\n\nSenior Enrollment Services Coordinator, Enrollment Management\n\n\n\n\nCaileigh Baker\n\n\n\nSenior Assistant Director of Admissions, Admissions\n\n\n\n\nPayam Bakhshi\n\n\n\nAssociate Professor, Construction Management, School of Management\n\n\n\n\nIuliana Balascuta\n\n\n\nAssociate Dean, School of Management\n\n\n\n\nNick Baldecchi\n\n\n\nHead Women's

In [58]:
temp_doc = professor_titles[0]
temp_doc.page_content = ''
temp_doc.metadata = ''

blank_document = temp_doc
blank_document

Document(page_content='', metadata='')

In [62]:
ptsplit = professor_titles.split('\n\n\n\n')
professor_split = []
for i in range(0, len(ptsplit)):
    professor_split.append(blank_document)
    professor_split[i].page_content = ptsplit[i]

AttributeError: 'list' object has no attribute 'split'

In [59]:
professor_digital_contact

[Document(page_content='\n\n\n\n\n      617-989-4182\n    \n\n\n\n\n\n\n      andersonl@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-5008\n    \n\n\n\n\n\n\n      andersone10@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-4296\n    \n\n\n\n\n\n\n      andersonk12@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-4457\n    \n\n\n\n\n\n\n      andresn@wit.edu\n    \n\n\n\n\n\n\n\n      anwaruddinm@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-4808\n    \n\n\n\n\n\n\n      arecyc@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-4537\n    \n\n\n\n\n\n\n      arevalod@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-4003\n    \n\n\n\n\n\n\n      armenisa@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-5341\n    \n\n\n\n\n\n\n      armstrongm2@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-4602\n    \n\n\n\n\n\n\n      askerneesee@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-4285\n    \n\n\n\n\n\n\n      assab@wit.edu\n    \n\n\n\n\n\n\n\n      austinwilksa@wit.edu\n    \n\n\n', metadata={'source': 'https://wit.edu/faculty-staff-

In [37]:
split_docs = [ ' '.join(i.page_content.split('\n')) for i in all_docs]

In [ ]:
split_docs

In [46]:
combined_unparsed_professor_info = []
for i in range(0, len(professor_titles)):

    title = professor_titles[i]
    digital = professor_digital_contact[i]
    physical = professor_physical_contact[i]

    combined_unparsed_professor_info.append(title)
    
    combined_unparsed_professor_info[i].page_content = title.page_content + digital.page_content + physical.page_content

In [47]:
combined_unparsed_professor_info

[Document(page_content='\n\nLenny Anderson\n\n\n\nAssociate Professor, Civil Engineering, School of Engineering\n\n\n\n\nEan Anderson\n\n\n\nStudent Success Advisor, Success Studio\n\n\n\n\nKayla Anderson\n\n\n\nStudent Success Advisor, Success Studio\n\n\n\n\nNicholas Andres\n\n\n\nAcademic Coordinator, School of Architecture and Design\n\n\n\n\nMohammed Anwaruddin\n\n\n\nAdjunct Instructor, Computer Science, School of Computing and Data Science\n\n\n\n\nChristina Arecy\n\n\n\nAssistant Director-Diversity, Engagement and Belonging, Diversity & Social Justice Programs\n\n\n\n\nDaniel Arevalo\n\n\n\nCampus Police Officer, Public Safety\n\n\n\n\nAndreas Armenis\n\n\n\nDirector of Precollege Programs, Enrollment Management\n\n\n\n\nMallory Armstrong\n\n\n\nAthletic Trainer, Athletics\n\n\n\n\nEverett Askerneese\n\n\n\nMaintenance A Worker, Physical Plant\n\n\n\n\nBen Assa\n\n\n\nData Architect, Digital Technology Services\n\n\n\n\nAngelique Austin-wilks\n\n\n\nTwo Semester Appointment, Sc

In [40]:
temp

['\n\nLenny Anderson\n\n\n\nAssociate Professor, Civil Engineering, School of Engineering',
 'Ean Anderson\n\n\n\nStudent Success Advisor, Success Studio',
 'Kayla Anderson\n\n\n\nStudent Success Advisor, Success Studio',
 'Nicholas Andres\n\n\n\nAcademic Coordinator, School of Architecture and Design',
 'Mohammed Anwaruddin\n\n\n\nAdjunct Instructor, Computer Science, School of Computing and Data Science',
 'Christina Arecy\n\n\n\nAssistant Director-Diversity, Engagement and Belonging, Diversity & Social Justice Programs',
 'Daniel Arevalo\n\n\n\nCampus Police Officer, Public Safety',
 'Andreas Armenis\n\n\n\nDirector of Precollege Programs, Enrollment Management',
 'Mallory Armstrong\n\n\n\nAthletic Trainer, Athletics',
 'Everett Askerneese\n\n\n\nMaintenance A Worker, Physical Plant',
 'Ben Assa\n\n\n\nData Architect, Digital Technology Services',
 'Angelique Austin-wilks\n\n\n\nTwo Semester Appointment, School of Computing and Data Science\n\n\n',
 '\n\nDanny Avelar\n\n\n\nMaintena

In [ ]:
split_docs = [ ' '.join(i.page_content.split('\n')) for i in all_docs]